1st Import Libraries and then the dataset

In [4]:
import pandas as pd
import numpy as np
!pip install black

ny_credits = pd.read_csv('https://raw.githubusercontent.com/hypoport/ep-dna-bewerberaufgabe/52645279ddc032955dabfaf43479da3e261b0709/sample.csv')
ny_credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175862 entries, 0 to 175861
Data columns (total 28 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   action_taken_name               175862 non-null  object 
 1   agency_name                     175862 non-null  object 
 2   applicant_ethnicity_name        175862 non-null  object 
 3   applicant_race_name_1           175862 non-null  object 
 4   applicant_sex_name              175862 non-null  object 
 5   applicant_income_000s           151344 non-null  float64
 6   co_applicant_ethnicity_name     175862 non-null  object 
 7   co_applicant_race_name_1        175862 non-null  object 
 8   co_applicant_sex_name           175862 non-null  object 
 9   county_name                     175377 non-null  object 
 10  denial_reason_name_1            23067 non-null   object 
 11  denial_reason_name_2            4650 non-null    object 
 12  denial_reason_na

Here I change parameters in order to be able to see the data according to my preferences

In [37]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

I will make a copy of the dataset not to re-write the original. ".copy" is also possible. I just prefer it this way!
And I add a lambda function to calculate the DTI (when possible) according to the data given.

In [38]:
ny_credits_1 = ny_credits.assign(DTI_nr = lambda x: x.hud_median_family_income / (x.applicant_income_000s*1000))
ny_credits_1

,action_taken_name,agency_name,applicant_ethnicity_name,applicant_race_name_1,applicant_sex_name,applicant_income_000s,co_applicant_ethnicity_name,co_applicant_race_name_1,co_applicant_sex_name,county_name,denial_reason_name_1,denial_reason_name_2,denial_reason_name_3,hoepa_status_name,lien_status_name,loan_purpose_name,loan_type_name,msamd_name,owner_occupancy_name,preapproval_name,property_type_name,purchaser_type_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,DTI_nr
0,Application denied by financial institution,Consumer Financial Protection Bureau,Not Hispanic or Latino,White,Female,176.00,No co-applicant,No co-applicant,No co-applicant,Chautauqua County,Collateral,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Refinancing,Conventional,NaN,Not owner-occupied as a principal dwelling,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,60000.00,235,2605.00,930.00,6.85,2979.00,0.34
1,Application withdrawn by applicant,Department of Housing and Urban Development,Not Hispanic or Latino,White,Male,53.00,No co-applicant,No co-applicant,No co-applicant,Erie County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,FHA-insured,"Buffalo, Cheektowaga, Niagara Falls - NY",Owner-occupied as a principal dwelling,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,69000.00,108,2053.00,821.00,33.58,4416.00,1.30
2,Loan originated,Consumer Financial Protection Bureau,Not Hispanic or Latino,White,Male,223.00,Not Hispanic or Latino,White,Female,Queens County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,Conventional,"New York, Jersey City, White Plains - NY, NJ",Owner-occupied as a principal dwelling,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,71300.00,659,497.00,387.00,49.96,1363.00,0.32
3,Loan originated,Consumer Financial Protection Bureau,Not Hispanic or Latino,Asian,Female,43.00,No co-applicant,No co-applicant,No co-applicant,Queens County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,Conventional,"New York, Jersey City, White Plains - NY, NJ",Owner-occupied as a principal dwelling,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,71300.00,103,251.00,558.00,32.09,1960.00,1.66
4,Loan originated,National Credit Union Administration,Not Hispanic or Latino,White,Male,72.00,No co-applicant,No co-applicant,No co-applicant,Steuben County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,Conventional,NaN,Owner-occupied as a principal dwelling,Preapproval was requested,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,60000.00,45,1514.00,1003.00,2.12,3391.00,0.83
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175857,Loan originated,Consumer Financial Protection Bureau,Not Hispanic or Latino,White,Female,118.00,No co-applicant,No co-applicant,No co-applicant,Nassau County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,Conventional,"Nassau County, Suffolk County - NY",Owner-occupied as a principal dwelling,Preapproval was requested,One-to-four family dwelling (other than manufa...,Affiliate institution,109000.00,300,1470.00,1337.00,17.11,4448.00,0.92
175858,Loan originated,Department of Housing and Urban Development,Not Hispanic or Latino,White,Female,27.00,No co-applicant,No co-applicant,No co-applicant,Monroe County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,FHA-insured,Rochester - NY,Owner-occupied as a principal dwelling,Preapproval was not requested,One-to-four family dwelling (other than manufa...,"Life insurance company, credit union, mortgage...",69700.00,

Here I am transforming categorical values into booleans so that I can sum them later when I need to.

In [39]:
def transform_denial_name(df):
    df['Collateral'] = df["denial_reason_name_1"].str.match(r'Collateral')
    df['Debt-to-income ratio'] = df["denial_reason_name_1"].str.match(r'Debt-to-income ratio')
    df['Credit history'] = df["denial_reason_name_1"].str.match(r'Credit history')
    df['Employment history'] = df["denial_reason_name_1"].str.match(r'Employment history')
    df['Other'] = df["denial_reason_name_1"].str.match(r'Other')
    df['Insufficient cash (downpayment, closing costs)'] = df["denial_reason_name_1"].str.match(r'Insufficient cash (downpayment, closing costs)')
    df['Credit application incomplete'] = df["denial_reason_name_1"].str.match(r'Credit application incomplete')
    df['Unverifiable information'] = df["denial_reason_name_1"].str.match(r'Unverifiable information')
    df['Mortgage insurance denied'] = df["denial_reason_name_1"].str.match(r'Mortgage insurance denied')
    return df

denial_to_num_df = transform_denial_name(ny_credits_1)
denial_to_num_df.head(50)

,action_taken_name,agency_name,applicant_ethnicity_name,applicant_race_name_1,applicant_sex_name,applicant_income_000s,co_applicant_ethnicity_name,co_applicant_race_name_1,co_applicant_sex_name,county_name,denial_reason_name_1,denial_reason_name_2,denial_reason_name_3,hoepa_status_name,lien_status_name,loan_purpose_name,loan_type_name,msamd_name,owner_occupancy_name,preapproval_name,property_type_name,purchaser_type_name,hud_median_family_income,loan_amount_000s,number_of_1_to_4_family_units,number_of_owner_occupied_units,minority_population,population,DTI_nr,Collateral,Debt-to-income ratio,Credit history,Employment history,Other,"Insufficient cash (downpayment, closing costs)",Credit application incomplete,Unverifiable information,Mortgage insurance denied
0,Application denied by financial institution,Consumer Financial Protection Bureau,Not Hispanic or Latino,White,Female,176.00,No co-applicant,No co-applicant,No co-applicant,Chautauqua County,Collateral,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Refinancing,Conventional,NaN,Not owner-occupied as a principal dwelling,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,60000.00,235,2605.00,930.00,6.85,2979.00,0.34,True,False,False,False,False,False,False,False,False
1,Application withdrawn by applicant,Department of Housing and Urban Development,Not Hispanic or Latino,White,Male,53.00,No co-applicant,No co-applicant,No co-applicant,Erie County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,FHA-insured,"Buffalo, Cheektowaga, Niagara Falls - NY",Owner-occupied as a principal dwelling,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,69000.00,108,2053.00,821.00,33.58,4416.00,1.30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Loan originated,Consumer Financial Protection Bureau,Not Hispanic or Latino,White,Male,223.00,Not Hispanic or Latino,White,Female,Queens County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,Conventional,"New York, Jersey City, White Plains - NY, NJ",Owner-occupied as a principal dwelling,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,71300.00,659,497.00,387.00,49.96,1363.00,0.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Loan originated,Consumer Financial Protection Bureau,Not Hispanic or Latino,Asian,Female,43.00,No co-applicant,No co-applicant,No co-applicant,Queens County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,Conventional,"New York, Jersey City, White Plains - NY, NJ",Owner-occupied as a principal dwelling,Not applicable,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,71300.00,103,251.00,558.00,32.09,1960.00,1.66,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Loan originated,National Credit Union Administration,Not Hispanic or Latino,White,Male,72.00,No co-applicant,No co-applicant,No co-applicant,Steuben County,NaN,NaN,NaN,Not a HOEPA loan,Secured by a first lien,Home purchase,Conventional,NaN,Owner-occupied as a principal dwelling,Preapproval was requested,One-to-four family dwelling (other than manufa...,Loan was not originated or was not sold in cal...,60000.00,45,1514.00,1003.00,2.12,3391.00,0.83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Loan purchased by the institution,Consumer Financial Protection Bureau,Not applicable,Not applicable,Not applicable,NaN,Not applicable,Not applicable,Not applicable,New York County,NaN,NaN,NaN,Not a HOEPA loan,Not applicable,Refinancing,Conventional,"New York, Jersey City, White Plains - NY, NJ",Not applicable,Not applicable,Multifamily dwelling,Loan was not originated or was not sold in cal...,71300.00,1650,NaN,686.00,37.49,6370.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Loan purchased by the institution,Consumer Financial Protection Bureau,Not applicable,Not applicable,Not applicable,77.00,Not applicable,Not applicable,N

In [40]:
denial_to_num_df.to_csv('reasons_loan_denied.csv')

Take an overview of your nummerical columns including the calculation of the DTI 
which is a creteria to deny loans if DTI > 43%

In [41]:
actions_in_numbers = (
    ny_credits_1[["action_taken_name","applicant_ethnicity_name", "applicant_income_000s", "hud_median_family_income", "loan_amount_000s", "DTI_nr", 'agency_name', 'denial_reason_name_1', 'denial_reason_name_2', 'denial_reason_name_3', 'lien_status_name', 'loan_purpose_name', 'loan_type_name']]
    .sort_values(by='DTI_nr', ascending= False)
)
actions_in_numbers.head(300)

,action_taken_name,applicant_ethnicity_name,applicant_income_000s,hud_median_family_income,loan_amount_000s,DTI_nr,agency_name,denial_reason_name_1,denial_reason_name_2,denial_reason_name_3,lien_status_name,loan_purpose_name,loan_type_name
171188,Application denied by financial institution,"Information not provided by applicant in mail,...",1.00,109000.00,10,109.00,Consumer Financial Protection Bureau,Debt-to-income ratio,NaN,NaN,Not secured by a lien,Home improvement,Conventional
103786,Application denied by financial institution,Not Hispanic or Latino,1.00,109000.00,626,109.00,Office of the Comptroller of the Currency,Debt-to-income ratio,NaN,NaN,Secured by a first lien,Home purchase,Conventional
28553,Application withdrawn by applicant,"Information not provided by applicant in mail,...",1.00,109000.00,350,109.00,Consumer Financial Protection Bureau,NaN,NaN,NaN,Secured by a first lien,Refinancing,Conventional
45706,Application denied by financial institution,Not Hispanic or Latino,1.00,109000.00,10,109.00,Consumer Financial Protection Bureau,Credit history,NaN,NaN,Secured by a subordinate lien,Home improvement,Conventional
10018,Application denied by financial institution,Not Hispanic or Latino,1.00,109000.00,10,109.00,Consumer Financial Protection Bureau,Unverifiable information,NaN,NaN,Not secured by a lien,Home improvement,Conventional
96997,File closed for incompleteness,Not Hispanic or Latino,1.00,109000.00,20,109.00,National Credit Union Administration,NaN,NaN,NaN,Secured by a first lien,Home improvement,Conventional
14209,Application denied by financial institution,Not Hispanic or Latino,1.00,109000.00,200,109.00,Consumer Financial Protection Bureau,Debt-to-income ratio,NaN,NaN,Secured by a first lien,Home improvement,Conventional
28219,Application withdrawn by applicant,Not Hispanic or Latino,1.00,109000.00,215,109.00,Department of Housing and Urban Development,NaN,NaN,NaN,Secured by a first lien,Home purchase,Conventional
77478,Application denied by financial institution,Not Hispanic or Latino,1.00,109000.00,200,109.00,Consumer Financial Protection Bureau,Debt-to-income ratio,Other,NaN,Secured by a first lien,Refinancing,Conventional
142316,Application denied by financial institution,Not Hispanic or Latino,1.00,109000.00,1200,109.00,Consumer Financial Protection Bureau,Debt-to-income ratio,NaN,NaN,Secured by a first lien,Refinancing,Conventional


In [42]:
actions_in_numbers.to_csv('searching_for_patterns_in_numbers.csv')

You will find many credits accepted inspite of score > 43%

This could be explained by either the type of credit or the credit institution, which loan fees may be cheaper




Here I created 2 functions to be able to filter data here and in the future.

In [43]:
def filter_field(df, f1, v1, f2, v2):
    return df[(f1 == v1) & (f2 == v2)]

def filter_field2(df, f1, v1, f2, v2):
    return df[(f1 == v1) & (f2 != v2)]

In [44]:
#Testing function
filter_field(actions_in_numbers, actions_in_numbers.action_taken_name, "Application denied by financial institution", actions_in_numbers.agency_name, "Office of the Comptroller of the Currency").count()


action_taken_name           2373
applicant_ethnicity_name    2373
applicant_income_000s       2258
hud_median_family_income    2373
loan_amount_000s            2373
DTI_nr                      2258
agency_name                 2373
denial_reason_name_1        2373
denial_reason_name_2         544
denial_reason_name_3         106
lien_status_name            2373
loan_purpose_name           2373
loan_type_name              2373
dtype: int64

In the following step I am replacing the values on the column "actions_taken" for "Credit" when the application was accepted, and "No_Credit" when the loan was denied.

Here it's not relevant whether the client took the credit or not in the end, because I am focusing on the reasons why the credit institution had approved or denied the applications. 

So I will drop those values. Besides, those categories ('Loan purchased by the institution'and 'Application withdrawn by applicant') have a lot of Nans, so it will also help with the cleaning!

In [45]:
#action_reasons = pd.Series(['Application denied by financial institution', 'File closed for incompleteness',
#                            'Preapproval request denied by financial institution', np.nan]).str.replace('No Credit','No Credit','No Credit', regex=True, inplace = True)
action_reasons= actions_in_numbers
action_reasons["action_taken_name"].replace(
    {"Application denied by financial institution": "No_credit", "File closed for incompleteness": "No_credit", 
     'Preapproval request denied by financial institution': "No_credit", "Loan originated": "Credit", 
     "Application approved but not accepted": "Credit"},
    inplace=True)


In [46]:
#To drop rows that contain any value in the list I 1st define values:
values_actions = ['Loan purchased by the institution', 'Application withdrawn by applicant']

#Now I can drop the rows containing any value in the defined list
action_reasons = action_reasons[action_reasons.action_taken_name.isin(values_actions) == False]

action_reasons.head()

,action_taken_name,applicant_ethnicity_name,applicant_income_000s,hud_median_family_income,loan_amount_000s,DTI_nr,agency_name,denial_reason_name_1,denial_reason_name_2,denial_reason_name_3,lien_status_name,loan_purpose_name,loan_type_name
171188,No_credit,"Information not provided by applicant in mail,...",1.00,109000.00,10,109.00,Consumer Financial Protection Bureau,Debt-to-income ratio,NaN,NaN,Not secured by a lien,Home improvement,Conventional
103786,No_credit,Not Hispanic or Latino,1.00,109000.00,626,109.00,Office of the Comptroller of the Currency,Debt-to-income ratio,NaN,NaN,Secured by a first lien,Home purchase,Conventional
45706,No_credit,Not Hispanic or Latino,1.00,109000.00,10,109.00,Consumer Financial Protection Bureau,Credit history,NaN,NaN,Secured by a subordinate lien,Home improvement,Conventional
10018,No_credit,Not Hispanic or Latino,1.00,109000.00,10,109.00,Consumer Financial Protection Bureau,Unverifiable information,NaN,NaN,Not secured by a lien,Home improvement,Conventional
96997,No_credit,Not Hispanic or Latino,1.00,109000.00,20,109.00,National Credit Union Administration,NaN,NaN,NaN,Secured by a first lien,Home improvement,Conventional


From the data I filtered, 71.5% of the cases the application has been accepted, and 28.5% has been denied

In [47]:
action_reasons.action_taken_name.str.count('No_credit').sum()

38690

In [48]:
action_reasons.action_taken_name.str.count('Credit').sum()


96725

# LET'S TAKE A LOOK AT THE DENIED CASES

In [49]:
No_credit = action_reasons.loc[action_reasons['action_taken_name'] == 'No_credit'].sort_values(by= "DTI_nr", ascending = True)
No_credit.drop(columns=['applicant_income_000s', 'hud_median_family_income'])
No_credit.head(300)

,action_taken_name,applicant_ethnicity_name,applicant_income_000s,hud_median_family_income,loan_amount_000s,DTI_nr,agency_name,denial_reason_name_1,denial_reason_name_2,denial_reason_name_3,lien_status_name,loan_purpose_name,loan_type_name
86791,No_credit,"Information not provided by applicant in mail,...",9999.00,71300.00,3850,0.01,Consumer Financial Protection Bureau,Collateral,NaN,NaN,Secured by a first lien,Refinancing,Conventional
125682,No_credit,Not Hispanic or Latino,9999.00,71300.00,4200,0.01,Consumer Financial Protection Bureau,Collateral,NaN,NaN,Secured by a subordinate lien,Home improvement,Conventional
137688,No_credit,Not Hispanic or Latino,9999.00,71300.00,441,0.01,Consumer Financial Protection Bureau,NaN,NaN,NaN,Secured by a first lien,Home purchase,Conventional
26324,No_credit,"Information not provided by applicant in mail,...",9999.00,71300.00,15000,0.01,Consumer Financial Protection Bureau,Collateral,NaN,NaN,Secured by a first lien,Refinancing,Conventional
48582,No_credit,"Information not provided by applicant in mail,...",9999.00,71300.00,1000,0.01,Consumer Financial Protection Bureau,Credit application incomplete,NaN,NaN,Secured by a first lien,Refinancing,Conventional
30314,No_credit,Not Hispanic or Latino,9999.00,71300.00,2000,0.01,Consumer Financial Protection Bureau,Credit application incomplete,NaN,NaN,Secured by a first lien,Home purchase,Conventional
76729,No_credit,Not Hispanic or Latino,9999.00,71300.00,200,0.01,Consumer Financial Protection Bureau,Collateral,NaN,NaN,Secured by a first lien,Home improvement,Conventional
54144,No_credit,Not Hispanic or Latino,9999.00,71300.00,250,0.01,Consumer Financial Protection Bureau,Credit history,NaN,NaN,Secured by a first lien,Home improvement,Conventional
108728,No_credit,"Information not provided by applicant in mail,...",8015.00,71300.00,350,0.01,National Credit Union Administration,NaN,NaN,NaN,Secured by a first lien,Refinancing,Conventional
55414,No_credit,Not Hispanic or Latino,8000.00,71300.00,365,0.01,Consumer Financial Protection Bureau,NaN,NaN,NaN,Secured by a first lien,Refinancing,Conventional


In [50]:
No_credit.to_csv('searching_for_No_credit_reasons.csv')

40% of the "No_credit" has no clear reason for denial. Only a 7% from that (1124 applications) do not count with a Secured lien, and could be explained by it.

In [51]:
Denial_unclear= No_credit.loc[No_credit['denial_reason_name_1'].isnull()].sort_values(by= "DTI_nr", ascending = False)
Denial_unclear


,action_taken_name,applicant_ethnicity_name,applicant_income_000s,hud_median_family_income,loan_amount_000s,DTI_nr,agency_name,denial_reason_name_1,denial_reason_name_2,denial_reason_name_3,lien_status_name,loan_purpose_name,loan_type_name
96997,No_credit,Not Hispanic or Latino,1.00,109000.00,20,109.00,National Credit Union Administration,NaN,NaN,NaN,Secured by a first lien,Home improvement,Conventional
55651,No_credit,Hispanic or Latino,1.00,71300.00,413,71.30,Department of Housing and Urban Development,NaN,NaN,NaN,Secured by a first lien,Refinancing,Conventional
99530,No_credit,"Information not provided by applicant in mail,...",1.00,71300.00,700,71.30,Department of Housing and Urban Development,NaN,NaN,NaN,Secured by a first lien,Home purchase,Conventional
40705,No_credit,Not Hispanic or Latino,1.00,71300.00,2800,71.30,Consumer Financial Protection Bureau,NaN,NaN,NaN,Secured by a first lien,Home improvement,Conventional
142172,No_credit,Not Hispanic or Latino,1.00,71300.00,203,71.30,Department of Housing and Urban Development,NaN,NaN,NaN,Secured by a first lien,Refinancing,Conventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...
175138,No_credit,Not Hispanic or Latino,NaN,109000.00,469,NaN,Department of Housing and Urban Development,NaN,NaN,NaN,Secured by a first lien,Refinancing,VA-guaranteed
175341,No_credit,Hispanic or Latino,NaN,71300.00,436,NaN,Department of Housing and Urban Development,NaN,NaN,NaN,Secured by a first lien,Refinancing,FHA-insured
175417,No_credit,Not Hispanic or Latino,NaN,71300.00,165,NaN,Department of Housing and Urban Development,NaN,NaN,NaN,Secured by a first lien,Refinancing,Conventional
175541,No_credit,Not Hispanic or Latino,NaN,71300.00,323,NaN,Department of Housing and Urban Development,NaN,NaN,NaN,Secured by a first lien,Refinancing,FHA-insured


The Etnicity has called my attention in the past. Is is there an obvious tendency of denial due to this?
Let's use the filter I created befofre:

In [52]:
filter_field(Denial_unclear, Denial_unclear.applicant_ethnicity_name, "Hispanic or Latino", Denial_unclear.action_taken_name, "No_credit").count()

action_taken_name           1165
applicant_ethnicity_name    1165
applicant_income_000s        978
hud_median_family_income    1153
loan_amount_000s            1165
DTI_nr                       966
agency_name                 1165
denial_reason_name_1           0
denial_reason_name_2           3
denial_reason_name_3          16
lien_status_name            1165
loan_purpose_name           1165
loan_type_name              1165
dtype: int64

Only 7% (1165 from 15623 unclear denial) are Latinos, this is not enough evidence for me to continue looking at that field.

No Credit Reasons: 

11% or more applications that have not been accepted do not count with a "lien"
To be more sure about this correlation a "correlation coefficient" could be calculated
This does not explain for certain though that there is causality

In [53]:
No_credit.lien_status_name.str.count('Not secured by a lien').sum()


4313

And another 7% doesnt have a lien

In [54]:
Denial_unclear.lien_status_name.str.count('Not secured by a lien').sum()

1124

# HERE WE FOCUSE ON APPROVED CREDITS

In [55]:
Credit = action_reasons.loc[action_reasons['action_taken_name'] == 'Credit']
Credit.drop(columns=['applicant_income_000s', 'applicant_ethnicity_name', 'hud_median_family_income', 'denial_reason_name_1', 'denial_reason_name_2', 'denial_reason_name_3'])

,action_taken_name,loan_amount_000s,DTI_nr,agency_name,lien_status_name,loan_purpose_name,loan_type_name
5333,Credit,362,109.00,Department of Housing and Urban Development,Secured by a first lien,Refinancing,FHA-insured
124731,Credit,184,82.70,Department of Housing and Urban Development,Secured by a first lien,Home purchase,Conventional
10736,Credit,200,71.30,Department of Housing and Urban Development,Secured by a first lien,Home purchase,Conventional
97000,Credit,2647,71.30,Office of the Comptroller of the Currency,Secured by a first lien,Refinancing,Conventional
50523,Credit,276,71.30,Consumer Financial Protection Bureau,Secured by a first lien,Refinancing,Conventional
...,...,...,...,...,...,...,...
175694,Credit,359,NaN,Department of Housing and Urban Development,Secured by a first lien,Refinancing,Conventional
175748,Credit,1165,NaN,Consumer Financial Protection Bureau,Secured by a subordinate lien,Home improvement,Conventional
175767,Credit,485,NaN,Consumer Financial Protection Bureau,Secured by a first lien,Refinancing,Conventional
175794,Credit,515,NaN,Department of Housing and Urban Development,Secured by a first lien,Refinancing,FHA-insured


In [56]:
Credit.to_csv('searching_for_patterns_in_approved_credits.csv')

94% of the approved credits count with a "Secured lien"

In [57]:
Credit.lien_status_name.str.count('Secured by a first lien').sum()

88427

In [58]:
Credit.lien_status_name.str.count('Secured by a subordinate lien').sum()


3408

6% of the approved credits do not have a lien

In [59]:
Credit.lien_status_name.str.count('Not secured by a lien').sum()

4890

In [60]:
#What about the DTI??
Credit.loc[Credit['DTI_nr'] >43].count()

action_taken_name           7
applicant_ethnicity_name    7
applicant_income_000s       7
hud_median_family_income    7
loan_amount_000s            7
DTI_nr                      7
agency_name                 7
denial_reason_name_1        0
denial_reason_name_2        0
denial_reason_name_3        0
lien_status_name            7
loan_purpose_name           7
loan_type_name              7
dtype: int64

Here I am creating a new dataframe to evaluate the specific cases that were approved despite of not having a lien.
Can I identify other factors???

In [61]:
No_lien = Credit.loc[Credit['lien_status_name'] == 'Not secured by a lien']
No_lien.drop(columns=['applicant_income_000s', 'hud_median_family_income', 'denial_reason_name_1', 'denial_reason_name_2', 'denial_reason_name_3'])

,action_taken_name,applicant_ethnicity_name,loan_amount_000s,DTI_nr,agency_name,lien_status_name,loan_purpose_name,loan_type_name
55511,Credit,Not Hispanic or Latino,4,63.00,National Credit Union Administration,Not secured by a lien,Home improvement,Conventional
63485,Credit,"Information not provided by applicant in mail,...",5,34.85,Consumer Financial Protection Bureau,Not secured by a lien,Home improvement,Conventional
54892,Credit,Hispanic or Latino,5,23.77,Consumer Financial Protection Bureau,Not secured by a lien,Home improvement,Conventional
159122,Credit,Not Hispanic or Latino,7,23.23,Consumer Financial Protection Bureau,Not secured by a lien,Home improvement,Conventional
98820,Credit,Not Hispanic or Latino,4,23.23,Consumer Financial Protection Bureau,Not secured by a lien,Home improvement,Conventional
...,...,...,...,...,...,...,...,...
161233,Credit,Not Hispanic or Latino,9,NaN,Federal Reserve System,Not secured by a lien,Home improvement,Conventional
161674,Credit,Not Hispanic or Latino,10,NaN,Federal Deposit Insurance Corporation,Not secured by a lien,Home improvement,Conventional
168776,Credit,Not Hispanic or Latino,5,NaN,Consumer Financial Protection Bureau,Not secured by a lien,Home improvement,Conventional
171098,Credit,Not Hispanic or Latino,15,NaN,Consumer Financial Protection Bureau,Not secured by a lien,Home improvement,Conventional


Here I am running my filter to quickly see if "Agency Name" could give me some insights on the above task

In [62]:
filter_field(No_lien, No_lien.agency_name, "National Credit Union Administration", No_lien.loan_type_name, "Conventional").count()

action_taken_name           1854
applicant_ethnicity_name    1854
applicant_income_000s       1849
hud_median_family_income    1845
loan_amount_000s            1854
DTI_nr                      1840
agency_name                 1854
denial_reason_name_1           0
denial_reason_name_2           0
denial_reason_name_3           0
lien_status_name            1854
loan_purpose_name           1854
loan_type_name              1854
dtype: int64

The Agency Name returns interesting values to continue the analysis on the field

Here I am transforming the categorical values into booleans to be able to sum them later

In [63]:
def transform_action_name(df):
    df['action_loan_denied'] = df["action_taken_name"].str.match(r'^Application\sdenied')
    df['action_application_withdrawn'] = df["action_taken_name"].str.match(r'^Application\swithdrawn')
    df['action_loan_originated'] = df["action_taken_name"].str.match(r'^Loan\soriginated')
    df['action_loan_purchased'] = df["action_taken_name"].str.match(r'^Loan\spurchased')
    df['action_loan_not_taken'] = df["action_taken_name"].str.match(r'^Application\sapproved.*but not accepted')  
    df['action_file_incomplete'] = df["action_taken_name"].str.match(r'File closed for incompletenes')
    df['action_preapproval_denied'] = df["action_taken_name"].str.match(r'Preapproval request denied by financial institution')

    return df

enriched_df = transform_action_name(ny_credits_1)

Here I want to get an overview of the actions taken grouped by agency

In [64]:
Credit_by_agency = (
    enriched_df[["agency_name", "action_loan_denied", "action_application_withdrawn", "action_loan_originated",
                 "action_loan_purchased", "action_loan_not_taken", "action_file_incomplete", "action_preapproval_denied"]]
    .groupby(["agency_name"]).sum())

Credit_by_agency

,action_loan_denied,action_application_withdrawn,action_loan_originated,action_loan_purchased,action_loan_not_taken,action_file_incomplete,action_preapproval_denied
agency_name,,,,,,,
Consumer Financial Protection Bureau,13564,5431,30626,16618,1501,3108,2
Department of Housing and Urban Development,11245,6713,30960,6622,2547,2345,0
Federal Deposit Insurance Corporation,872,589,4070,23,337,283,0
Federal Reserve System,754,297,2680,29,200,142,0
National Credit Union Administration,3197,1995,14048,9,587,543,1
Office of the Comptroller of the Currency,2373,740,8598,1381,571,261,0


Let's see the loan's average amount per agency.
Further analysis can be done in terms of: 
    Which company gave more credits, and which one gives more money per credit.
    This can give an insight towards what type of interest/motivation each institution may have to give a loan

In [65]:
Loans_by_agency = (
    ny_credits_1[["agency_name", "loan_amount_000s"]]
    .groupby(["agency_name"])
    .agg({"loan_amount_000s": ["sum", "count"]})
)
Loans_by_agency.assign(ratio=lambda r: r[('loan_amount_000s', 'sum')]/r[('loan_amount_000s', 'count')]).sort_values(by="ratio")

loan_amount_000s         ratio
                                                         sum  count       
agency_name                                                               
National Credit Union Administration                 2963754  20380 145.42
Office of the Comptroller of the Currency            3485847  13924 250.35
Department of Housing and Urban Development         17151998  60432 283.82
Federal Reserve System                               1218084   4102 296.95
Federal Deposit Insurance Corporation                2303391   6174 373.08
Consumer Financial Protection Bureau                31489580  70850 444.45

# This analysis is a bit more isolated and falls on the limit of the time i had to complete the task. This is why I didn't go deeper into this topic.
Nevertheless, I would like to have a very brief look here at the applicants' sex.
Which sex has applied the most? Can we find any tendency in the resolution based on this creteria?

In [66]:
def transform_sex_name(df):
    df['Female'] = df["applicant_sex_name"].str.match(r'Female')
    df['Male'] = df["applicant_sex_name"].str.match(r'Male')

    return df

ny_credits_1 = transform_sex_name(ny_credits_1)

#loan decision by sex
loans_by_sex = (
    ny_credits_1[["action_taken_name", "Female", "Male"]]
    .groupby(["action_taken_name"]).sum()
)
loans_by_sex

,Female,Male
action_taken_name,,
Application approved but not accepted,1586,3663
Application denied by financial institution,10232,18592
Application withdrawn by applicant,4559,9610
File closed for incompleteness,1834,4075
Loan originated,27415,56031
Loan purchased by the institution,2404,4965
Preapproval request denied by financial institution,0,3


In [67]:
loans_by_sex.to_csv('loans_by_sex.csv')

# THE END

In [68]:
# Supporting research not used:
enriched_df[enriched_df.applicant_ethnicity_name == 'Hispanic or Latino']['applicant_ethnicity_name'].count()

10015

There are 10015 applicants who are latinos